In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import  accuracy_score
import sqlite3

In [2]:
international_matches = pd.read_csv("international_matches.csv")

In [3]:
international_matches.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,1993-08-08,Bolivia,Uruguay,South America,South America,59,22,0,0,3,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-08-08,Brazil,Mexico,South America,North America,8,14,0,0,1,...,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-08-08,Ecuador,Venezuela,South America,South America,35,94,0,0,5,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-08-08,Guinea,Sierra Leone,Africa,Africa,65,86,0,0,1,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-08-08,Paraguay,Argentina,South America,South America,67,5,0,0,1,...,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
international_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23921 entries, 0 to 23920
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   date                           23921 non-null  object 
 1   home_team                      23921 non-null  object 
 2   away_team                      23921 non-null  object 
 3   home_team_continent            23921 non-null  object 
 4   away_team_continent            23921 non-null  object 
 5   home_team_fifa_rank            23921 non-null  int64  
 6   away_team_fifa_rank            23921 non-null  int64  
 7   home_team_total_fifa_points    23921 non-null  int64  
 8   away_team_total_fifa_points    23921 non-null  int64  
 9   home_team_score                23921 non-null  int64  
 10  away_team_score                23921 non-null  int64  
 11  tournament                     23921 non-null  object 
 12  city                           23921 non-null 

In [5]:
international_matches.isnull().sum().sort_values(ascending=False)

away_team_mean_defense_score     16357
home_team_mean_defense_score     16134
away_team_mean_midfield_score    15942
away_team_goalkeeper_score       15826
home_team_mean_midfield_score    15759
away_team_mean_offense_score     15609
home_team_goalkeeper_score       15542
home_team_mean_offense_score     15411
country                              0
home_team_result                     0
shoot_out                            0
neutral_location                     0
date                                 0
home_team                            0
tournament                           0
away_team_score                      0
home_team_score                      0
away_team_total_fifa_points          0
home_team_total_fifa_points          0
away_team_fifa_rank                  0
home_team_fifa_rank                  0
away_team_continent                  0
home_team_continent                  0
away_team                            0
city                                 0
dtype: int64

Посмотрим как закончились игры.

In [6]:
international_matches['home_team_result'].value_counts().sort_values(ascending=False)

Win     11761
Lose     6771
Draw     5389
Name: home_team_result, dtype: int64

In [7]:
def transform(result):
    if result == "Win":
        return 1
    elif result == "Lose":
        return -1
    else:
        return 0
international_matches['home_team_result'] = international_matches['home_team_result'].map(transform)

Создание переменных, которые добавляют оценку атаки (полузащита + нападение) и оценку защиты (вратарь + защита).

In [8]:
international_matches['home_attack'] = international_matches['home_team_mean_midfield_score'] + international_matches['home_team_mean_offense_score']
international_matches['home_defense'] = international_matches['home_team_goalkeeper_score'] + international_matches['home_team_mean_defense_score']

international_matches['away_attack'] = international_matches['away_team_mean_midfield_score'] + international_matches['away_team_mean_offense_score']
international_matches['away_defense'] = international_matches['away_team_goalkeeper_score'] + international_matches['away_team_mean_defense_score']


Создание переменной с общим рейтингом матча(рейтинг домашней команды минус рейтинг команда гостей).

In [9]:
international_matches['rank_difference'] = international_matches['home_team_fifa_rank'] - international_matches['away_team_fifa_rank']

Удаление лишней информации.

In [10]:
international_matches = international_matches.drop(['home_team_continent', 'away_team_continent', 'city','country', 
              'shoot_out', 'home_team_mean_midfield_score', 'home_team_mean_offense_score', 
              'home_team_goalkeeper_score', 'home_team_mean_defense_score', 
              'away_team_mean_midfield_score', 'away_team_mean_offense_score', 
              'away_team_goalkeeper_score',  'away_team_mean_defense_score'], axis=1)

In [11]:
international_matches

,date,home_team,away_team,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,tournament,neutral_location,home_team_result,home_attack,home_defense,away_attack,away_defense,rank_difference
0,1993-08-08,Bolivia,Uruguay,59,22,0,0,3,1,FIFA World Cup qualification,False,1,NaN,NaN,NaN,NaN,37
1,1993-08-08,Brazil,Mexico,8,14,0,0,1,1,Friendly,False,0,NaN,NaN,NaN,NaN,-6
2,1993-08-08,Ecuador,Venezuela,35,94,0,0,5,0,FIFA World Cup qualification,False,1,NaN,NaN,NaN,NaN,-59
3,1993-08-08,Guinea,Sierra Leone,65,86,0,0,1,0,Friendly,False,1,NaN,NaN,NaN,NaN,-21
4,1993-08-08,Paraguay,Argentina,67,5,0,0,1,3,FIFA World Cup qualification,False,-1,NaN,NaN,NaN,NaN,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23916,2022-06-14,Moldova,Andorra,180,153,932,1040,2,1,UEFA Nations League,False,1,NaN,NaN,NaN,NaN,27
23917,2022-06-14,Liechtenstein,Latvia,192,135,895,1105,0,2,UEFA Nations League,False,-1,NaN,NaN,NaN,NaN,57
23918,2022-06-14,Chile,Ghana,28,60,1526,1387,0,0,Kirin Cup,True,-1,154.9,154.5,154.2,149.5,-32
23919,2022-06-14,Japan,Tunisia,23,35,1553,1499,0,3,Kirin Cup,False,-1,152.5,148.2,146.3,NaN,-12


In [12]:
international_matches = international_matches[international_matches['tournament']== 'FIFA World Cup']

Добавим столбец, который предполагает победителя матча в зависимости от общего рейтинга матча.

In [13]:
def victory(base):
    if base > 0:
        return -1
    else:
        return 1
international_matches['victory'] = international_matches['rank_difference'].map(victory)

C:\Users\ASLAN\AppData\Local\Temp\ipykernel_9492\4192877686.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  international_matches['victory'] = international_matches['rank_difference'].map(victory)


In [14]:
international_matches

,date,home_team,away_team,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,tournament,neutral_location,home_team_result,home_attack,home_defense,away_attack,away_defense,rank_difference,victory
385,1994-06-17,Germany,Bolivia,1,43,0,0,1,0,FIFA World Cup,True,1,NaN,NaN,NaN,NaN,-42,1
386,1994-06-17,Spain,Korea Republic,5,37,0,0,2,2,FIFA World Cup,True,0,NaN,NaN,NaN,NaN,-32,1
387,1994-06-18,Colombia,Romania,17,7,0,0,1,3,FIFA World Cup,True,-1,NaN,NaN,NaN,NaN,10,-1
388,1994-06-18,Italy,Republic of Ireland,4,14,0,0,0,1,FIFA World Cup,True,-1,NaN,NaN,NaN,NaN,-10,1
389,1994-06-18,USA,Switzerland,23,12,0,0,1,1,FIFA World Cup,False,0,NaN,NaN,NaN,NaN,11,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20445,2018-07-07,Russia,Croatia,49,12,1758,2036,2,2,FIFA World Cup,False,-1,159.5,157.5,165.1,165.8,37,-1
20446,2018-07-10,France,Belgium,2,5,2164,2124,1,0,FIFA World Cup,True,1,172.5,172.0,173.1,172.5,-3,1
20447,2018-07-11,Croatia,England,12,7,2036,2099,2,1,FIFA World Cup,True,1,165.1,165.8,166.1,164.8,5,-1
20448,2018-07-14,Belgium,England,5,7,2124,2099,2,0,FIFA World Cup,True,1,173.1,172.5,166.1,164.8,-2,1


In [15]:
acc1 = accuracy_score(y_true=international_matches['home_team_result'], y_pred=international_matches['victory'])
print('Acc: {:.4f}'.format(acc1))

Acc: 0.5648


In [16]:
international_matches.dtypes

date                            object
home_team                       object
away_team                       object
home_team_fifa_rank              int64
away_team_fifa_rank              int64
home_team_total_fifa_points      int64
away_team_total_fifa_points      int64
home_team_score                  int64
away_team_score                  int64
tournament                      object
neutral_location                  bool
home_team_result                 int64
home_attack                    float64
home_defense                   float64
away_attack                    float64
away_defense                   float64
rank_difference                  int64
victory                          int64
dtype: object

In [17]:
conn = sqlite3.connect("world_championship.db")
cursor = conn.cursor()
query = ("""CREATE TABLE IF NOT EXISTS conn(
   date Text,
   home_team Text,
   away_team Text,
   home_team_fifa_rank Integer,
   away_team_fifa_rank Integer,
   home_team_total_fifa_points Integer,
   away_team_total_fifa_points Integer,
   home_team_score Integer,
   away_team_score Integer,
   tournament Text,
   neutral_location Real,
   home_team_result Integer,
   home_attack Real,
   home_defense Real,
   away_attack Real,
   away_defense Real,
   rank_difference Integer,
   victory Integer)
""")
cursor.execute(query)

In [18]:
international_matches.to_sql("conn",conn,if_exists='replace',index=False)
conn.commit()

cursor.execute("SELECT * FROM conn;")    
one_result = cursor.fetchall()
print(one_result)


conn.close()

[('1994-06-17', 'Germany', 'Bolivia', 1, 43, 0, 0, 1, 0, 'FIFA World Cup', 1, 1, None, None, None, None, -42, 1), ('1994-06-17', 'Spain', 'Korea Republic', 5, 37, 0, 0, 2, 2, 'FIFA World Cup', 1, 0, None, None, None, None, -32, 1), ('1994-06-18', 'Colombia', 'Romania', 17, 7, 0, 0, 1, 3, 'FIFA World Cup', 1, -1, None, None, None, None, 10, -1), ('1994-06-18', 'Italy', 'Republic of Ireland', 4, 14, 0, 0, 0, 1, 'FIFA World Cup', 1, -1, None, None, None, None, -10, 1), ('1994-06-18', 'USA', 'Switzerland', 23, 12, 0, 0, 1, 1, 'FIFA World Cup', 0, 0, None, None, None, None, 11, -1), ('1994-06-19', 'Belgium', 'Morocco', 27, 28, 0, 0, 1, 0, 'FIFA World Cup', 1, 1, None, None, None, None, -1, 1), ('1994-06-19', 'Cameroon', 'Sweden', 24, 10, 0, 0, 2, 2, 'FIFA World Cup', 1, 0, None, None, None, None, 14, -1), ('1994-06-19', 'Norway', 'Mexico', 6, 16, 0, 0, 1, 0, 'FIFA World Cup', 1, 1, None, None, None, None, -10, 1), ('1994-06-20', 'Brazil', 'Russia', 3, 19, 0, 0, 2, 0, 'FIFA World Cup', 1, 1,